# WhisperX Cloud - Minimal Install

Quick setup: Mamba -> pip install whisperx -> run server

Steps:
1. Set GPU: Runtime -> Change runtime type -> GPU
2. Paste ngrok token in Step 5
3. Run All

In [ ]:
# Step 1: Install Mamba and create environment
import os, sys, subprocess

ENV_PATH = '/content/conda-envs/whisperx' if 'google.colab' in sys.modules else '/kaggle/working/conda-envs/whisperx' if os.path.exists('/kaggle') else os.path.expanduser('~/conda-envs/whisperx')

print("Installing Mamba...")
if not os.path.exists(os.path.expanduser('~/miniforge3/bin/mamba')):
    !wget -q -O /tmp/miniforge.sh https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
    !bash /tmp/miniforge.sh -b -p ~/miniforge3

MAMBA = os.path.expanduser('~/miniforge3/bin/mamba')

print(f"Creating env: {ENV_PATH}")
if os.path.exists(ENV_PATH):
    !{MAMBA} remove -p {ENV_PATH} --all -y -q 2>/dev/null || true

!{MAMBA} create -p {ENV_PATH} -c conda-forge -y python=3.10 ffmpeg git pip
print("Env created!")

In [ ]:
# Step 2: pip install whisperx
PYTHON = f"{ENV_PATH}/bin/python"

print("Installing whisperx...")
print("This may take a few minutes...")

# Do not upgrade pip to avoid dependency conflicts
!{PYTHON} -m pip install whisperx pyngrok

print("\nDone!")

In [ ]:
# Step 3: Verify
print("Verifying...")
for pkg in ['torch', 'whisperx', 'fastapi']:
    r = subprocess.run([PYTHON, '-c', f'import {pkg}; print("OK")'], capture_output=True)
    print(f"  {'OK' if r.returncode == 0 else 'FAIL'} {pkg}")

import json
with open('.conda_python_path', 'w') as f:
    json.dump({'python_path': PYTHON, 'env_prefix': ENV_PATH}, f)

In [ ]:
# Step 4: Download server
import urllib.request
url = "https://raw.githubusercontent.com/infinite-gaming-studio/VideoLingo/main/whisperx_cloud/whisperx_server.py"
if not os.path.exists('whisperx_server.py'):
    urllib.request.urlretrieve(url, 'whisperx_server.py')
    print("Downloaded")
else:
    print("Exists")

In [ ]:
# Step 5: Start server with ngrok
NGROK_TOKEN = ""  # <-- PASTE YOUR NGROK TOKEN HERE

if not NGROK_TOKEN:
    raise ValueError("Please paste your ngrok token above! Get it from https://dashboard.ngrok.com/get-started/your-authtoken")

import threading, time
sys.path.insert(0, f"{ENV_PATH}/lib/python3.10/site-packages")
from pyngrok import ngrok, conf

# Set token and connect
conf.get_default().auth_token = NGROK_TOKEN
public_url = ngrok.connect(8000, "http")
print(f"URL: {public_url}")
print(f"\nCopy to VideoLingo: {public_url}")

# Start server
def start():
    subprocess.Popen([PYTHON, 'whisperx_server.py', '--port', '8000'])
threading.Thread(target=start, daemon=True).start()
time.sleep(5)
print("\nServer running!")

In [ ]:
# Step 7: Keep alive (DO NOT STOP THIS CELL!)
# Re-import in case this cell runs separately
from pyngrok import ngrok

print(f"Running: {public_url}")
print("\nDO NOT stop this cell!\nPress Ctrl+C to stop")
try:
    while True: time.sleep(60)
except KeyboardInterrupt:
    ngrok.disconnect(public_url)
    print("\nStopped")